In [ ]:
!pip install sentence-transformers

In [ ]:
import pandas as pd
import numpy as np

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
data = pd.read_csv("ratings_Electronics.csv")

In [ ]:
data.columns

Index(['AKM1MP6P0OYPR', '0132793040', '5.0', '1365811200'], dtype='object')

In [ ]:
data.columns = ['reviewerID','asin','rating','unixReviewTime']

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 669711 entries, 0 to 669710
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   reviewerID      669711 non-null  object 
 1   asin            669710 non-null  object 
 2   rating          669710 non-null  float64
 3   unixReviewTime  669710 non-null  float64
dtypes: float64(2), object(2)
memory usage: 20.4+ MB


In [ ]:
data['rating'] = data['rating'].fillna(data['rating'].median())
data['unixReviewTime'] = data['unixReviewTime'].fillna(data['unixReviewTime'].median())
data['asin'] = data['asin'].fillna(data['asin'].mode()[0])

In [ ]:
data.head(5)

,reviewerID,asin,rating,unixReviewTime
0,A2CX7LUOHB2NDG,0321732944,5.0,1.341101e+09
1,A2NWSAGRHCP8N5,0439886341,1.0,1.367194e+09
2,A2WNBOD3WNDNKT,0439886341,3.0,1.374451e+09
3,A1GI0U4ZRJA8WN,0439886341,1.0,1.334707e+09
4,A1QGNMC6O1VW39,0511189877,5.0,1.397434e+09


In [ ]:
data.sort_values('unixReviewTime',inplace=True)

In [ ]:
product_text = (
    data.groupby('asin')['rating']
    .apply(lambda x: ' '.join(x.astype(str)))
    .reset_index()
)

In [ ]:
product_text.head(5)

,asin,rating
0,0321732944,5.0
1,0439886341,1.0 1.0 3.0
2,0511189877,5.0 5.0 5.0 5.0 2.0 5.0
3,0528881469,2.0 4.0 4.0 3.0 5.0 5.0 1.0 4.0 2.0 1.0 5.0 2....
4,0558835155,3.0


In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
product_embeddings = model.encode(
    product_text['rating'].tolist(),
    show_progress_bar = True
)

Batches:   0%|          | 0/1277 [00:00<?, ?it/s]

In [ ]:
product_embeddings[0]

array([-5.80696436e-03, -2.14195270e-02,  6.34998679e-02,  3.64311673e-02,
        3.99447381e-02, -8.35469812e-02,  1.41171287e-04, -1.61803234e-02,
       -2.70199906e-02, -9.62778646e-03,  1.82037037e-02, -1.73210725e-02,
       -1.06642708e-01,  1.86772272e-02, -3.19645144e-02,  3.49140391e-02,
       -2.74431868e-03, -1.04727827e-01, -6.59150928e-02,  1.06050251e-02,
        2.11787280e-02, -1.45758362e-02,  8.36123992e-03,  1.39095392e-02,
        6.06155954e-02, -2.95073036e-02, -4.54181209e-02,  3.13606039e-02,
       -4.39516008e-02, -1.09221786e-01, -5.16771935e-02,  7.44093675e-03,
        4.75584855e-03, -6.21154122e-02, -1.02584802e-01, -6.95975944e-02,
        7.10507780e-02, -1.45380860e-02, -5.96506260e-02,  8.99725929e-02,
        1.28168557e-02, -6.06057979e-03, -1.00825448e-02,  4.24109325e-02,
        2.73601487e-02, -5.72948437e-03,  1.65273563e-03,  3.93967479e-02,
        5.93955070e-03, -7.18948606e-04, -2.06977557e-02, -2.95474045e-02,
       -5.98657988e-02, -

In [ ]:
product_embeddings.shape

(40834, 384)

In [24]:
# User Intent Modeling

def get_user_intent(user_id,last_n=3):
  user_data = (
      data[data['reviewerID'] == user_id]
      .sort_values('unixReviewTime')
      .tail(last_n)
  )

  if len(user_data) == 0:
    return None

  text = ' '.join(user_data['rating'].astype(str))
  return model.encode(text)

In [25]:
# Behaviour Drift Weighting

def apply_drift(similarity_score,decay=0.7):
  weights = np.linspace(decay,1,len(similarity_score))
  return similarity_score * weights

In [27]:
# Recommendation function

def recommend_products(user_id,top_k=5):
  user_vec = get_user_intent(user_id)

  if user_vec is None:
    return "No data for this user"

  sims = cosine_similarity([user_vec],product_embeddings)[0]
  sims = apply_drift(sims)

  top_idx = sims.argsort()[-top_k:][::-1]
  return product_text.iloc[top_idx][['asin']]

In [28]:
sample_user = data['reviewerID'].iloc[0]
sample_user

'A2ZUNCI13N2OUL'

In [29]:
recommend_products(sample_user)

,asin
40810,B0009K4V7E
40779,B0009JVS44
40777,B0009JUOVM
40771,B0009JS42S
40763,B0009JR5O6
